# [Module 1.3] Horovod 분산 훈련

### 본 워크샵의 모든 노트북은 `conda_python3` 여기에서 작업 합니다.

이 노트북은 아래와 같은 작업을 합니다.
- 1. 환경 셋업
- 2. 세이지 메이크 로컬 모드 훈련
- 3. SageMaker Host Mode 로 훈련
- 4. 모델 아티펙트 저장
 
---

### 참고
- [Horovod Official Git](https://github.com/horovod/horovod)
- [Horovod with PyTorch](https://horovod.readthedocs.io/en/stable/pytorch.html)
- [Pytorch Horovod Example on SageMaker ](https://github.com/aws/amazon-sagemaker-examples/tree/main/sagemaker-python-sdk/pytorch_horovod_mnist)

# 1. 환경 셋업

## 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

In [2]:
### 커스텀 라이브러리
import config 

버킷 및 폴더(prefix) 설정

In [3]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

use_default_bucket = True
if use_default_bucket:
    bucket = sagemaker_session.default_bucket()
else:
    bucket = '<Type your bucket>'
    
prefix = "NCFModel"



# 2. 세이지 메이크 로컬 모드 훈련
#### 로컬의 GPU, CPU 여부로 instance_type 결정

In [4]:
import os
import subprocess


try:
    if subprocess.call("nvidia-smi") == 0:
        ## Set type to GPU if one is present
        instance_type = "local_gpu"
    else:
        instance_type = "local"        
except:
    pass

print("Instance type = " + instance_type)

Instance type = local_gpu


## 2.1. 스크립트 모드의 코드 작성 방법
- ![script_mode_example](img/script_mode_example.png)

## 2.2.훈련 코드 확인
- 아래의 코드는 전형적인 스크립트 모드의 코드 작성 방법을 따르고 있습니다.
- 훈련 함수는 `from train_lib import train` 로서 이전 노트북의 **[세이지 메이커 없이]** 작성한 스크래치 버전에서 사용한 훈련 함수와 동일 합니다.


In [5]:
# train_code = 'src/train_horovod.py'
# !pygmentize {train_code}

## 2.3. 로컬에 있는 데이타 세트의 위치를 지정 합니다.

In [6]:
local_inputs = config.main_path
print("local_inputs: ", local_inputs)

local_inputs:  ../data/


In [7]:
local_inputs = {'train': f'file://{local_inputs}',
          'test': f'file://{local_inputs}'}

## 2.4. 로컬 모드로 훈련 실행
- 아래의 두 라인이 로컬모드로 훈련을 지시 합니다.
```python
    instance_type=instance_type, # local_gpu or local 지정
    session = sagemaker.LocalSession(), # 로컬 세션을 사용합니다.
```

In [8]:
hyperparameters = {'epochs': 1, 
                   'lr': 0.001,
                   'batch_size': 256,
                   'top_k' : 10,
                   'dropout' : 0.0,
                   'factor_num' : 32,
                   'num_layers' : 3,
                   'num_ng' : 4,
                   'test_num_ng' : 99,   
                   'log_interval': 500
                    }  

In [9]:
import torch
num_gpu = torch.cuda.device_count()
#num_gpu =1

In [10]:
from sagemaker.pytorch import PyTorch
import os
import subprocess


local_estimator = PyTorch(
    entry_point="train_horovod.py",    
    source_dir='src',    
    role=role,
    framework_version='1.8.1',
    py_version='py3',
    instance_count=1,
    instance_type=instance_type, # local_gpu or local 지정
    session = sagemaker.LocalSession(), # 로컬 세션을 사용합니다.
    hyperparameters= hyperparameters,
    distribution={
        "mpi": {
            "enabled": True,
            "processes_per_host": num_gpu,
            "custom_mpi_options": "--NCCL_DEBUG INFO"
        }
    }        
)

local_estimator.fit(local_inputs)

Creating gpqqd5bis7-algo-1-zo1k2 ... 
Creating gpqqd5bis7-algo-1-zo1k2 ... done
Attaching to gpqqd5bis7-algo-1-zo1k2
gpqqd5bis7-algo-1-zo1k2 | 2022-05-27 10:12:43,712 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
gpqqd5bis7-algo-1-zo1k2 | 2022-05-27 10:12:43,807 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
gpqqd5bis7-algo-1-zo1k2 | 2022-05-27 10:12:43,810 sagemaker_pytorch_container.training INFO     Invoking user training script.
gpqqd5bis7-algo-1-zo1k2 | 2022-05-27 10:12:44,019 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
gpqqd5bis7-algo-1-zo1k2 | /opt/conda/bin/python3.6 -m pip install -r requirements.txt
gpqqd5bis7-algo-1-zo1k2 | Requirement already satisfied: nvidia-ml-py3==7.352 in /opt/conda/lib/python3.6/site-packages (from -r requirements.txt (line 1)) (7.352.0)
gpqqd5bis7-algo-1-zo1k2 | Collecting pandas==0.24.2
gpqqd5bis7-algo-1-zo1k2 |   Downloading

# 3. SageMaker Host Mode 로 훈련
- instance_type, session 을 수정 합니다.
- 입력 데이터를 inputs로서 S3 의 경로를 제공합니다.
- wait=False 로 지정해서 async 모드로 훈련을 실행합니다. 
- 실행 경과는 아래의 cifar10_estimator.logs() 에서 확인 합니다.

## 3.1. 데이터 세트를 S3에 업로드


In [11]:
s3_data_loc = sagemaker_session.upload_data(path=config.main_path, bucket=bucket, 
                                       key_prefix=f"{prefix}/data")
print("s3_data_loc: ", s3_data_loc)

s3_data_loc:  s3://sagemaker-us-east-1-057716757052/NCFModel/data


In [12]:
! aws s3 ls {s3_data_loc} --recursive

2022-05-24 12:16:00     128039 NCFModel/data/.ipynb_checkpoints/ml-1m.test-checkpoint.rating
2022-05-24 12:16:00   20982911 NCFModel/data/.ipynb_checkpoints/ml-1m.train-checkpoint.rating
2022-05-27 10:14:43    2891424 NCFModel/data/ml-1m.test.negative
2022-05-27 10:14:42     128039 NCFModel/data/ml-1m.test.rating
2022-05-27 10:14:42   20982911 NCFModel/data/ml-1m.train.rating
2022-05-24 12:16:00      57986 NCFModel/data/ml-1m.train.rating-small
2022-05-21 07:03:58   27404899 NCFModel/data/pinterest-20.test.negative
2022-05-21 07:03:54     807267 NCFModel/data/pinterest-20.test.rating
2022-05-21 07:03:55   21138451 NCFModel/data/pinterest-20.train.rating


## 3.2. 훈련 및 테스트 데이터를 S3 로 지정

In [13]:
s3_inputs = {
            'train': f'{s3_data_loc}',
            'test': f'{s3_data_loc}'
            }

print("s3_inputs: \n", s3_inputs)

s3_inputs: 
 {'train': 's3://sagemaker-us-east-1-057716757052/NCFModel/data', 'test': 's3://sagemaker-us-east-1-057716757052/NCFModel/data'}


## 3.3. 실험 세팅

### 실험(Experiment) 세팅
- Amazon SageMaker 실험은 기계 학습 실험을 구성, 추적, 비교 및 평가할 수 있는 Amazon SageMaker 의 기능입니다
- 상세 사항은 개발자 가이드 참조 하세요. --> [Amazon SageMaker 실험을 통한 Machine Learning 관리](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/experiments.html)
- sagemaker experiment는 추가적인 패키지를 설치하여야 합니다. 1_Setup_Environment 가 실행이 안되었다고 하면, `!pip install --upgrade sagemaker-experiments` 를 통해 설치 해주세요.
- 여기서는 boto3 API를 통해서 실험을 생성합니다. SageMaker Python SDK를 통해서도 가능합니다.


### 실험 생성

In [14]:
# !pip install --upgrade sagemaker-experiments
import boto3
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

from datetime import datetime

sm_client = boto3.client('sagemaker')


# 설험에 대한 이름을 생성 합니다.
experiment_name = prefix + '-single-train'

# 실험이 존재하지 않으면 생성하고, 그렇지 않으면 지나갑니다.
try:
    response = sm_client.describe_experiment(ExperimentName=experiment_name)
    print(f"Experiment:{experiment_name} already exists")    
    
except:
    response = sm_client.create_experiment(
        ExperimentName = experiment_name,
        Description = 'Experiment for NCF',
    )
    print(f"Experiment:{experiment_name} is created")        


Experiment:NCFModel-single-train already exists


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


### 하이퍼 파라미터 세팅
### 간단한 실험을 위해서 epochs 값을 조절해서 실행 시간을 조정 하세요.

In [15]:
host_hyperparameters = {'epochs': 5, 
                       'lr': 0.001,
                       'batch_size': 256,
                       'top_k' : 10,
                       'dropout' : 0.0,
                       'factor_num' : 32,
                       'num_layers' : 3,
                       'num_ng' : 4,
                       'test_num_ng' : 99,   
                       'log_interval': 500                        
                    }  

### 시도(Trial) 생성

In [16]:
from datetime import datetime
# 시도 이름 생성
ts = datetime.now().strftime('%Y-%m-%d-%H-%M-%S-%f')
trial_name = experiment_name + f"-{ts}"

# 1개의 실험 안에 시도를 생성함.
response = sm_client.create_trial(
    ExperimentName = experiment_name,
    TrialName = trial_name,
)    

# 실험 설정: 실험 이름, 시도 이름으로 구성
experiment_config = {
    'ExperimentName' : experiment_name,
    'TrialName' : trial_name,
    "TrialComponentDisplayName" : 'Training',
}    



## 3.4 훈련 실행


### 훈련 메트릭을 CloudWatch 에서 보기
- 개발자 가이드
    - [Monitor and Analyze Training Jobs Using Amazon CloudWatch ](https://docs.amazonaws.cn/en_us/sagemaker/latest/dg/training-metrics.html#define-train-metrics)

In [17]:
metric_definitions=[
       {'Name': 'HR', 'Regex': 'HR=(.*?);'},
       {'Name': 'NDCG', 'Regex': 'NDCG=(.*?);'},
       {'Name': 'Loss', 'Regex': 'Loss=(.*?);'}    
    ]


## [중요] 테스트에는 instance_type 을  'ml.p3.2xlarge' 해주세요
- 현재 계정에 리소스 제한이 있어서 'ml.p3.8xlarge', 'ml.p3.16xlarge' 는 사용 못할 수 있습니다.

In [18]:
# instance_type = 'ml.p3.16xlarge'
instance_type = 'ml.p3.2xlarge' 

if instance_type == 'ml.p3.16xlarge':
    num_gpu = 8
elif instance_type == 'ml.p3.8xlarge':
    num_gpu = 4    
elif instance_type == 'ml.p3.2xlarge':
    num_gpu = 1
else:
    num_gpu = 1

In [19]:
from sagemaker.pytorch import PyTorch

host_estimator = PyTorch(
    entry_point="train_horovod.py",    
    source_dir='src',    
    role=role,
    framework_version='1.8.1',
    py_version='py3',
    instance_count=1,
    instance_type=instance_type,
    session = sagemaker.Session(), # 세이지 메이커 세션
    hyperparameters=host_hyperparameters,
    metric_definitions = metric_definitions,
    distribution={
        "mpi": {
            "enabled": True,
            "processes_per_host": num_gpu,
            "custom_mpi_options": "--NCCL_DEBUG INFO"
        }
    }        

    
    
)
host_estimator.fit(s3_inputs, 
                   experiment_config = experiment_config, # 실험 설정 제공                   
                   wait=False)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: pytorch-training-2022-05-27-10-14-43-786


In [20]:
%%time

host_estimator.logs()

2022-05-27 10:14:44 Starting - Starting the training job...ProfilerReport-1653646484: InProgress
...
2022-05-27 10:15:41 Starting - Preparing the instances for training...............
2022-05-27 10:18:07 Downloading - Downloading input data...
2022-05-27 10:18:27 Training - Downloading the training image.....................
2022-05-27 10:22:17 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-05-27 10:22:21,197 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-05-27 10:22:21,272 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-05-27 10:22:21,280 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-05-27 10:22:21,896 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip insta

# 4. 실험 결과 보기


위의 실험한 결과를 확인 합니다.
- 각각의 훈련잡의 시도에 대한 훈련 사용 데이터, 모델 입력 하이퍼 파라미터, 모델 평가 지표, 모델 아티펙트 결과 위치 등의 확인이 가능합니다.
- **아래의 모든 내용은 SageMaker Studio 를 통해서 직관적으로 확인이 가능합니다.**

In [21]:
from sagemaker.analytics import ExperimentAnalytics
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 5
pd.options.display.max_colwidth = 50

search_expression = {
    "Filters": [
        {
            "Name": "DisplayName",
            "Operator": "Equals",
            "Value": "Training",
        }
    ],
}


trial_component_analytics = ExperimentAnalytics(
    sagemaker_session= sagemaker_session,
    experiment_name= experiment_name,
    search_expression=search_expression,
)

trial_component_analytics.dataframe()

,TrialComponentName,DisplayName,SourceArn,SageMaker.ImageUri,SageMaker.InstanceCount,SageMaker.InstanceType,SageMaker.VolumeSizeInGB,batch_size,dropout,epochs,factor_num,log_interval,lr,num_layers,num_ng,sagemaker_container_log_level,sagemaker_job_name,sagemaker_program,sagemaker_region,sagemaker_submit_directory,test_num_ng,top_k,test - MediaType,test - Value,train - MediaType,...,BCEWithLogitsLoss_output_0_GLOBAL - Max,BCEWithLogitsLoss_output_0_GLOBAL - Avg,BCEWithLogitsLoss_output_0_GLOBAL - StdDev,BCEWithLogitsLoss_output_0_GLOBAL - Last,BCEWithLogitsLoss_output_0_GLOBAL - Count,Loss - Min,Loss - Max,Loss - Avg,Loss - StdDev,Loss - Last,Loss - Count,HR - Min,HR - Max,HR - Avg,HR - StdDev,HR - Last,HR - Count,NDCG - Min,NDCG - Max,NDCG - Avg,NDCG - StdDev,NDCG - Last,NDCG - Count,SageMaker.ModelArtifact - MediaType,SageMaker.ModelArtifact - Value
0,pytorch-training-2022-05-27-10-24-16-588-aws-t...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,763104351884.dkr.ecr.us-east-1.amazonaws.com/p...,1.0,ml.p3.8xlarge,30.0,256.0,0.0,10.0,32.0,500.0,0.001,3.0,4.0,20.0,"""pytorch-training-2022-05-27-10-24-16-588""","""train_horovod.py""","""us-east-1""","""s3://sagemaker-us-east-1-057716757052/pytorch...",99.0,10.0,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,pytorch-training-2022-05-27-10-14-43-786-aws-t...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,763104351884.dkr.ecr.us-east-1.amazonaws.com/p...,1.0,ml.p3.16xlarge,30.0,256.0,0.0,10.0,32.0,500.0,0.001,3.0,4.0,20.0,"""pytorch-training-2022-05-27-10-14-43-786""","""train_horovod.py""","""us-east-1""","""s3://sagemaker-us-east-1-057716757052/pytorch...",99.0,10.0,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,...,0.693148,0.693081,0.000046,0.693047,8.0,0.158554,0.375369,0.233753,0.04766,0.26189,80.0,0.593,0.697,0.6718,0.031668,0.681,10.0,0.335,0.420,0.400100,0.026635,0.412,10.0,NaN,s3://sagemaker-us-east-1-057716757052/pytorch-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,pytorch-training-2022-05-22-03-52-23-671-aws-t...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,763104351884.dkr.ecr.us-east-1.amazonaws.com/p...,1.0,ml.p3.2xlarge,30.0,256.0,0.0,10.0,32.0,NaN,0.001,3.0,4.0,20.0,"""pytorch-training-2022-05-22-03-52-23-671""","""train.py""","""us-east-1""","""s3://sagemaker-us-east-1-057716757052/pytorch...",99.0,10.0,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,...,0.691033,0.234167,0.048146,0.252895,390.0,NaN,NaN,NaN,NaN,NaN,NaN,0.630,0.696,0.6825,0.019609,0.687,10.0,0.366,0.421,0.410200,0.017080,0.417,10.0,NaN,s3://sagemaker-us-east-1-057716757052/pytorch-...
16,pytorch-training-2022-05-22-03-16-01-173-aws-t...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,763104351884.dkr.ecr.us-east-1.amazonaws.com/p...,1.0,ml.p3.2xlarge,30.0,256.0,0.0,3.0,32.0,NaN,0.001,3.0,4.0,20.0,"""pytorch-training-2022-05-22-03-16-01-173""","""train.py""","""us-east-1""","""s3://sagemaker-us-east-1-057716757052/pytorch...",99.0,10.0,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,...,0.693058,0.266076,0.057190,0.250393,117.0,NaN,NaN,NaN,NaN,NaN,NaN,0.625,0.685,0.6600,0.031225,0.685,3.0,0.358,0.405,0.384667,0.024132,0.405,3.0,NaN,s3://sagemaker-us-east-1-057716757052/pytorch-...


### 모델 평가 지표에 순서에 따른 시도 보기
- 아래는 모델 평가 지표에 따른 순서로 보여주기 입니다.

In [22]:

trial_component_training_analytics = ExperimentAnalytics(
    sagemaker_session= sagemaker_session,
    experiment_name= experiment_name,
    search_expression=search_expression,
    sort_by="metrics.HR.max",        
    sort_order="Descending",
    metric_names=["HR"],    
    parameter_names=["epochs", "batch_size", "dropout", "factor_num", 
                     "lr", "num_layers" ,"num_ng", "test_num_ng", "top_k",
                    ],
)

trial_component_training_analytics.dataframe()

,TrialComponentName,DisplayName,SourceArn,batch_size,dropout,epochs,factor_num,lr,num_layers,num_ng,test_num_ng,top_k,HR - Min,HR - Max,HR - Avg,HR - StdDev,HR - Last,HR - Count,test - MediaType,test - Value,train - MediaType,train - Value,SageMaker.DebugHookOutput - MediaType,SageMaker.DebugHookOutput - Value,SageMaker.ModelArtifact - MediaType,SageMaker.ModelArtifact - Value,Trials,Experiments
0,pytorch-training-2022-05-27-10-14-43-786-aws-t...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,256.0,0.0,10.0,32.0,0.001,3.0,4.0,99.0,10.0,0.593,0.697,0.6718,0.031668,0.681,10.0,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,s3://sagemaker-us-east-1-057716757052/,NaN,s3://sagemaker-us-east-1-057716757052/pytorch-...,[NCFModel-single-train-2022-05-27-10-14-43-519...,[NCFModel-single-train]
1,pytorch-training-2022-05-22-03-52-23-671-aws-t...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,256.0,0.0,10.0,32.0,0.001,3.0,4.0,99.0,10.0,0.630,0.696,0.6825,0.019609,0.687,10.0,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,s3://sagemaker-us-east-1-057716757052/,NaN,s3://sagemaker-us-east-1-057716757052/pytorch-...,[NCFModel-single-train-2022-05-22-03-52-16-349...,[NCFModel-single-train]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,pytorch-training-2022-05-26-13-59-29-387-aws-t...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,256.0,0.0,10.0,32.0,0.001,3.0,4.0,99.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,s3://sagemaker-us-east-1-057716757052/,NaN,s3://sagemaker-us-east-1-057716757052/pytorch-...,[NCFModel-single-train-2022-05-26-13-36-15-959...,[NCFModel-single-train]
16,pytorch-training-2022-05-26-13-36-29-023-aws-t...,Training,arn:aws:sagemaker:us-east-1:057716757052:train...,256.0,0.0,10.0,32.0,0.001,3.0,4.0,99.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,s3://sagemaker-us-east-1-057716757052/NCFModel...,None,s3://sagemaker-us-east-1-057716757052/,NaN,s3://sagemaker-us-east-1-057716757052/pytorch-...,[NCFModel-single-train-2022-05-26-13-36-15-959...,[NCFModel-single-train]


# 5. 모델 아티펙트 저장
- S3 에 저장된 모델 아티펙트를 저장하여 추론시 사용합니다.

In [23]:
horovod_artifact_path = host_estimator.model_data
print("horovod_artifact_path: ", horovod_artifact_path)

%store horovod_artifact_path

horovod_artifact_path:  s3://sagemaker-us-east-1-057716757052/pytorch-training-2022-05-27-10-14-43-786/output/model.tar.gz
Stored 'horovod_artifact_path' (str)


기타 변수 저장

In [24]:
%store bucket 
%store prefix

Stored 'bucket' (str)
Stored 'prefix' (str)
